In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import pandas as pd
import seaborn as sns

In [ ]:
class BasicStats:  

    def __init__(self, class_data):
        self.object = class_data  
        self.mois_possibles = ["Janvier", "Février", "Mars", "Avril", "Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"]
        self.month = None
 
    @property
    def data(self):
        return self.object.data

    @property
    def agence(self):
        agences = self.data.index.unique()
        assert len(agences) == 1, f"Le jeu de données contient plusieurs agences au lieu d'en contenir une seule {agences}"
        return self.data.index[0]

    @property
    def year(self):
        annees = self.data["date_heure_operation"].dt.year.unique()
        assert len(annees) == 1, f"Le jeu de données contient plusieurs années au lieu d'en contenir une seule {annees}"
        return annees[0]

    def nb_obs_jour(self):
        nb_jours_ouvres = self.data["jour"].nunique()
        print(f"Nombre de jours ouvrés dans l'année {self.year} :", nb_jours_ouvres)
        nb_obs_jour = self.data.groupby("jour")["montant_operation"].count()
        nb_retraits_jours = self.data[self.data["débit"] != 0].groupby("jour")["débit"].count()
        nb_versements_jours = self.data[self.data["crédit"] != 0].groupby("jour")["crédit"].count()
        nombre = {"j_ouvres": nb_jours_ouvres, "obs_j" : nb_obs_jour, "retraits_j" : nb_retraits_jours, "versements_j": nb_versements_jours}
        return nombre

    def vals_seuil_nb_obs(self):
        data = self.nb_obs_jour()
        min_obs_jour = data["obs_j"].min()
        max_obs_jour = data["obs_j"].max()
        moy_obs_jour = data["obs_j"].mean()
        min_nb_retrait_jour = data["retraits_j"].min()
        max_nb_retrait_jour = data["retraits_j"].max()
        moy_nb_retrait_jour = data["retraits_j"].mean()
        median_nb_retrait_jour = data["retraits_j"].median()
        std_nb_retrait_jour = data["retraits_j"].std()
        min_nb_versement_jour = data["versements_j"].min()
        max_nb_versement_jour = data["versements_j"].max()
        moy_nb_versement_jour = data["versements_j"].mean()
        median_nb_versement_jour = data["versements_j"].median()
        std_nb_versement_jour = data["versements_j"].std()
        print("Moyenne des opérations (versements / retraits) par jour :", moy_obs_jour)
        print("Plus petit nombre d'opérations observées en un jour :", min_obs_jour)
        print("Plus grand nombre d'opérations observées en un jour :", max_obs_jour)
        print("Moyenne du nombre de retraits par jour: ", moy_nb_retrait_jour)
        print("Médiane du nombre de retraits par jour: ", median_nb_retrait_jour)
        print("Ecart-type du nombre de retraits par jour: ", std_nb_retrait_jour)
        print("Plus petit nombre de retraits observés en un jour :", min_nb_retrait_jour)
        print("Plus grand nombre de retraits observés en un jour :", max_nb_retrait_jour)
        print("Plus petit nombre de versements observés en un jour :", min_nb_versement_jour)
        print("Plus grand nombre de versements observés en un jour :", max_nb_versement_jour)
        print("Moyenne du nombre de versements par jour: ", moy_nb_versement_jour)
        print("Médiane du nombre de versements par jour: ", median_nb_versement_jour)
        print("Ecart-type du nombre de versements par jour: ", std_nb_versement_jour)
        summary_obs = {"min_obs_j": min_obs_jour, "max_obs_j": max_obs_jour, "moy_obs_j": moy_obs_jour,
                       "min_nb_retraits_j": min_nb_retrait_jour, "max_nb_retraits_j": max_nb_retrait_jour,
                       "moy_nb_retraits_j": moy_nb_retrait_jour, "median_nb_retraits_j": median_nb_retrait_jour,
                       "std_nb_retraits_j": std_nb_retrait_jour, "min_nb_versements_j": min_nb_versement_jour,
                       "max_nb_versements_j": max_nb_versement_jour, "moy_nb_versements_j": moy_nb_versement_jour,
                       "median_nb_versements_j": median_nb_versement_jour, "std_nb_versements_j": std_nb_versement_jour}
        return summary_obs 


    def boxplot_nb_operations(self):
        nb = self.nb_obs_jour()
        nb_retrait_versement = pd.DataFrame({"nb_retraits_moyens": nb["retraits_j"],
                                             "nb_versements_moyens": nb["versements_j"]}).fillna(0)
        sns.boxplot(data = nb_retrait_versement)
        plt.title(f"Distribution du nombre de retraits et versements par jour pour l'agence {self.agence}")
        plt.ylabel("Nombre de transactions")
        plt.show()

    def montants_obs_jour(self):
        stat_versement = self.data[self.data["débit"] != 0].groupby("jour")["débit"]
        stat_retrait = self.data[self.data["crédit"] != 0].groupby("jour")["crédit"]
        moy_retraits_jour = stat_retrait.mean()
        median_retraits_jour = stat_retrait.median()
        std_deviation_retrait_jour = stat_retrait.std()
        moy_versements_jour = stat_versement.mean()
        median_versements_jour = stat_versement.median()
        std_deviation_versement_jour = stat_versement.std()
        montants = {"moy_retraits_j": moy_retraits_jour, "moy_versements_j": moy_versements_jour, "med_retraits_j":median_retraits_jour,
                    "med_versements_j": median_versements_jour, "std_versements_j": std_deviation_versement_jour, "std_retraits_j": std_deviation_retrait_jour}
        return montants

    def vals_seuil_montants(self):
        montant = self.montants_obs_jour()
        retraits_moy_j = montant["moy_retraits_j"].mean()
        versement_moy_j = montant["moy_versements_j"].mean()
        print(f"Retrait moyen par jour pour l'agence {self.agence}: ", retraits_moy_j)
        print(f"Versement moyen par jour pour l'agence {self.agence}: ", versement_moy_j)

    def boxplot_moy_montant_operations(self):
        montant = self.montants_obs_jour()
        montant_retrait_versement = pd.DataFrame({"montant_retraits_moyens": montant["moy_retraits_j"],
                                                  "montant_versements_moyens": montant["moy_versements_j"]})
        sns.boxplot(data = montant_retrait_versement)
        plt.title(f"Distribution du montant moyen des retraits et versements par jour pour l'agence {self.agence}")
        plt.ylabel("Montant moyen des transactions")
        plt.show()

    def boxplot_median_montant_operations(self):
        median = self.montants_obs_jour()
        median_retrait_versement = pd.DataFrame({"median_montant_retraits": median["med_retraits_j"],
                                                  "median_montant_versements": median["med_versements_j"]})
        sns.boxplot(data = median_retrait_versement)
        plt.title(f"Distribution du montant médian des retraits et versements par jour pour l'agence {self.agence}")
        plt.ylabel("Montant médian des transactions")
        plt.show()

    def quantiles_retraits(self):
        médiane = self.data[self.data["débit"] != 0]["débit"].quantile(0.5)
        quantile_90 = self.data[self.data["débit"] != 0]["débit"].quantile(0.90)
        quantile_98 = self.data[self.data["débit"] != 0]["débit"].quantile(0.98)
        quantile_99 = self.data[self.data["débit"] != 0]["débit"].quantile(0.99)
        quantile_999 = self.data[self.data["débit"] != 0]["débit"].quantile(0.999)
        print("Médiane des retraits: ", médiane)
        print("Quantile 0.90 des retraits: ", quantile_90)
        print("Quantile 0.98 des retraits: ", quantile_98)
        print("Quantile 0.99 des retraits: ", quantile_99)
        print("Quantile 0.999 des retraits: ", quantile_999)
        dict_quantile_versements = {"quant_50": médiane, "quant_90": quantile_90, "quant_98": quantile_98, "quant_99": quantile_99, "quant_999": quantile_999}
        return dict_quantile_versements
        

    def quantiles_versements(self):
        médiane = self.data[self.data["crédit"] != 0]["crédit"].quantile(0.5)
        quantile_90 = self.data[self.data["crédit"] != 0]["crédit"].quantile(0.90)
        quantile_98 = self.data[self.data["crédit"] != 0]["crédit"].quantile(0.98)
        quantile_99 = self.data[self.data["crédit"] != 0]["crédit"].quantile(0.99)
        quantile_999 = self.data[self.data["crédit"] != 0]["crédit"].quantile(0.999)
        print("Médiane des retraits: ", médiane)
        print("Quantile 0.90 des retraits: ", quantile_90)
        print("Quantile 0.98 des retraits: ", quantile_98)
        print("Quantile 0.99 des retraits: ", quantile_99)
        print("Quantile 0.999 des retraits: ", quantile_999)
        dict_quantile_retraits = {"quant_50": médiane, "quant_90": quantile_90, "quant_98": quantile_98, "quant_99": quantile_99, "quant_999": quantile_999}
        return dict_quantile_retraits
        

    def define_quantile(self, value : float):
        if not (0 < value < 1):
            raise ValueError("La valeur entrée doit être strictement comprise entre 0 et 1")
        else:
            new_quantile_retrait = self.data[self.data["débit"] != 0]["débit"].quantile(value)
            new_quantile_versement = self.data[self.data["crédit"] != 0]["crédit"].quantile(value)
            print(f"Quantile {value} pour les retraits: ", new_quantile_retrait)
            print(f"Quantile {value} pour les versements: ", new_quantile_versement)

    def distribution_retraits(self, nb_bins = 50):
        plt.figure(figsize = (14,12))
        sns.histplot(self.data[self.data["débit"] != 0]["débit"], bins = nb_bins, kde = False, color = 'red')
        plt.title(f"Distribution des retraits pour l'agence {self.agence} (en {self.year})")
        plt.xlabel("Montant retiré")
        plt.ylabel("Nombre de retraits")
        plt.grid(True)
        plt.show()

    def distribution_versements(self, nb_bins = 50):
        plt.figure(figsize = (14,12))
        sns.histplot(self.data[self.data["crédit"] != 0]["crédit"], bins = nb_bins, kde = False, color = 'green')
        plt.title(f"Distribution des versements pour l'agence {self.agence} (en {self.year})")
        plt.xlabel("Montant versé")
        plt.ylabel("Nombre de versements")
        plt.grid(True)
        plt.show()

    def custom_distrib_retraits(self, value_sup : float, value_inf = None, nb_bins = 40):
        if not value_inf:
            plt.figure(figsize = (14,12))
            sns.histplot(self.data[(self.data["débit"] != 0) & (self.data["débit"] <= value_sup)]["débit"], bins = nb_bins, kde = False, color = 'orange')
            plt.title(f"Distribution des retraits pour l'agence {self.agence} (en {self.year})")
            plt.xlabel("Montant retiré")
            plt.ylabel(f"Nombre de retraits inférieurs à {value_sup}")
            plt.grid(True)
            plt.show()
        else:
            plt.figure(figsize = (14,12))
            sns.histplot(self.data[(self.data["débit"] != 0) & (self.data["débit"] <= value_sup) & (self.data["débit"] >= value_inf)]["débit"], bins = nb_bins, kde = False, color = 'orange')
            plt.title(f"Distribution des retraits pour l'agence {self.agence} (en {self.year})")
            plt.xlabel("Montant retiré")
            plt.ylabel(f"Nombre de retraits compris entre {value_inf} et {value_sup}")
            plt.grid(True)
            plt.show()

    def custom_distrib_versements(self, value_sup, value_inf = None, nb_bins = 40):
        if not value_inf:
            plt.figure(figsize = (14,12))
            sns.histplot(self.data[(self.data["crédit"] != 0) & (self.data["crédit"] <= value_sup)]["crédit"], bins = nb_bins, kde = False, color = 'blue')
            plt.title(f"Distribution des versements pour l'agence {self.agence} (en {self.year})")
            plt.xlabel("Montant versé")
            plt.ylabel(f"Nombre de versements inférieurs à {value_sup}")
            plt.grid(True)
            plt.show()
        else:
            plt.figure(figsize = (14,12))
            sns.histplot(self.data[(self.data["crédit"] != 0) & (self.data["crédit"] <= value_sup) & (self.data["crédit"] >= value_inf)]["crédit"], bins = nb_bins, kde = False, color = 'blue')
            plt.title(f"Distribution des versements pour l'agence {self.agence} (en {self.year})")
            plt.xlabel("Montant versé")
            plt.ylabel(f"Nombre de versements compris entre {value_inf} et {value_sup}")
            plt.grid(True)
            plt.show()

    def plot_cumsum_montants_mois(self, mois):
        self.month = self.data[self.data["date_heure_operation"].dt.month == mois]
        self.month = self.month.sort_values("date_heure_operation")
        self.month = self.month.copy()
        self.month["somme_cumule_montants"] = self.month["montant_operation"].cumsum()
        vals_fin_jour = self.month.groupby("jour")["somme_cumule_montants"].last().reset_index()
        plt.figure(figsize = (12,10))
        plt.plot(vals_fin_jour["jour"], vals_fin_jour["somme_cumule_montants"], marker = 'o')
        plt.title(f"Evolution des montants (versements - retraits) à la fin de chaque journée pour l'agence {self.agence} - {self.mois_possibles[mois-1]} {self.year}")
        plt.xlabel("Jour")
        plt.ylabel("Montant à la fin de la journée")
        plt.xticks(rotation = 45)
        plt.show()

    def seuil_debut_jour(self, mois, seuil = 0):
        self.month = self.data[self.data["date_heure_operation"].dt.month == mois]
        self.month = self.month.sort_values("date_heure_operation")
        self.month = self.month.copy()
        self.month["cumsum_montants"] = self.month.groupby("jour")["montant_operation"].cumsum()
        df_cumule_jour = self.month.groupby("jour")["cumsum_montants"].last().reset_index()
        df_cumule_jour["cumsum_montants"] = df_cumule_jour["cumsum_montants"] + seuil
        plt.figure(figsize = (12,10))
        plt.plot(df_cumule_jour["jour"], df_cumule_jour["cumsum_montants"], marker = 'o')
        plt.title(f"Montants cumulés (versements - retraits) par jour, en supposant un seuil {seuil} pour l'agence {self.agence} - {self.mois_possibles[mois-1]} {self.year}")
        plt.xlabel("Journée")
        plt.ylabel(f"Montant cumulé sur la journée en partant d'un seuil {seuil}")
        plt.xticks(rotation = 45)
        plt.grid(True)
        plt.show()

    def pire_debit(self, mois):
        self.month = self.data[self.data["date_heure_operation"].dt.month == mois]
        self.month = self.month.sort_values("date_heure_operation")
        self.month = self.month.copy()
        self.month["somme_cumule_montants"] = self.month.groupby("jour")["montant_operation"].cumsum()
        pire_debit = self.month["somme_cumule_montants"].min()
        print(f"Pire débit atteint par l'agence {self.agence} en {self.mois_possibles[mois-1]} {self.year}: ", pire_debit)
        return pire_debit

    def jour_critique(self, mois):
        pire_debit = self.pire_debit()
        self.month = self.data[self.data["date_heure_operation"].dt.month == mois]
        self.month = self.month.sort_values("date_heure_operation")
        self.month = self.month.copy()
        self.month["somme_cumule_montants"] = self.month.groupby("jour")["montant_operation"].cumsum()
        data_jour_critique = self.month.loc[self.month["somme_cumule_montants"] == pire_debit]
        print(f"Jour critique pour l'agence {self.agence} en {self.mois_possibles[mois-1]} {self.year}: ", data_jour_critique)

    def nb_clients(self, mois = None):
        if not mois:
            nb_clients = self.data["identifiant_client"].nunique()
            print(f"Nombre de clients pour l'agence {self.agence} à l'année {self.year}: ", nb_clients)
        else:
            nb_clients = self.data[self.data["date_heure_operation"].dt.month == mois]["identifiant_client"].nunique()
            print(f"Nombre de clients pour l'agence {self.agence} en {self.mois_possibles[mois-1]} {self.year}: ", nb_clients)
        return nb_clients



# Méthodes pour les retraits importants, susceptibles de faire tomber l'agence en rupture: 

    def set_seuil_imp(self, seuil):
        self.seuil = seuil

    def retraits_imps(self):   # A modifier pour prendre en compte le cas où l'agence n'aurait vu aucun retrait de ce type
        retraits_imp = self.data[self.data["débit"] >= self.seuil].copy()
        dict_retraits_imp = {jour : [len(groupe), list(groupe["débit"])] 
                             for jour, groupe in retraits_imp.groupby("jour")}
        if retraits_imp.empty:
            print(f"Aucun retrait important détecté (au sens de la valeur seuil fournie {self.seuil})")
            return {}
        else:
            nb_retraits_imp = len(dict_retraits_imp)
            print(f"Nombre de retraits importants (supérieurs à {self.seuil}) pour l'agence {self.agence} en {self.year}: ", nb_retraits_imp)
            return dict_retraits_imp

    def visu_retraits_imp(self, quantite = 10):
        dict_requis = self.retraits_imps()
        plot_retraits_imps = pd.DataFrame([
            {'date': pd.to_datetime(date), 'somme_retraits_imps_jour': sum(montants), "nombre_retraits_imp_jour": nb}
            for date, (nb,montants) in dict_requis.items()
        ])
        plot_retraits_imps = plot_retraits_imps.sort_values('date')
        fig, ax1 = plt.subplots(figsize = (15,13))
        color1 = 'tab:blue'
        ax1.set_xlabel('Date des retraits')
        ax1.set_ylabel("Somme des montants des retraits (en MDH)", color = color1)
        ax1.plot(plot_retraits_imps["date"], plot_retraits_imps["somme_retraits_imps_jour"], color = color1, marker = 'o', label = 'Montants retraits')
        ax1.tick_params(axis = 'y', labelcolor = color1)
        ax2 = ax1.twinx()
        color2 = 'tab:red'
        ax2.set_ylabel("Nombre de retraits dans la journée", color = color2)
        ax2.plot(plot_retraits_imps["date"], plot_retraits_imps["nombre_retraits_imp_jour"], color = color2, marker = 's', linestyle = '--', label = 'Nombre retraits')
        ax2.tick_params(axis = 'y', labelcolor = color2)
        plt.title(f"Evolution de la somme et du nombre des retraits journaliers importants (supérieurs à {self.seuil} MAD) pour l'agence {self.agence} sur l'année {self.year}")
        fig.autofmt_xdate()
        plt.show()

    def freq_retraits_imps(self):
        freq_imp = (self.data[self.data["débit"] != 0]["débit"] > self.seuil).mean()*100
        print(f"La fréquence des retraits supérieurs à {self.seuil} pour l'agence {self.agence} en {self.year} est de: ", freq_imp)
        return freq_imp 
    
    def count_freq_above(self, threshold):
        jours_ouvres = self.nb_obs_jour()["j_ouvres"]
        above = self.data[self.data["débit"]>=threshold]
        count = above.shape[0]
        freq = count / jours_ouvres
        print("Nombre de retraits qui dépassent la valeur fixée: ", count)
        print("Fréquence de retraits qui dépassent la valeur fixée: ", freq)
        return count, freq

    def meshgrid_threshold(self):
        mesh = [threshold for threshold in np.linspace(100000,1500000,100000)]
        for threshold in mesh:
            self.count_freq_above(threshold)
        print("Fin de l'exploration")

    def custom_meshgrid_threshold(self, limit_1 : int, limit_2 : int, jump : int):
        mesh = [threshold for threshold in np.linspace(limit_1,limit_2,jump)]
        for threshold in mesh:
            self.count_freq_above(threshold)
        print("Fin de l'exploration")


# Méthodes pour lancer une première analyse globale et pour construire un DataFrame nécessaire à la clusterisation:

    def analyse_preliminaire_data(self):
        self.object.nb_agences_annees_dataset()
        self.nb_clients()
        self.vals_seuil_nb_obs()
        self.vals_seuil_montants()
        self.quantiles_retraits()
        self.quantiles_versements()
        seuil = int(input("Entrez un seuil de retrait important en fonction des quantiles précédents: "))
        self.def_seuil_imp(seuil)
        self.visu_retraits_imp()
        self.freq_retraits_imps()
        self.boxplot_nb_operations()
        self.boxplot_moy_montant_operations()
        self.boxplot_median_montant_operations()
        self.distribution_retraits()
        self.distrib_retraits_imp()
        month = int(input("Entrez une valeur entre 1 et 12 qui représente le mois correspondant: "))
        self.nb_clients(month)
        self.pire_debit(month)
        self.jour_critique(month)
        self.plot_cumsum_montants_mois(month)
        self.seuil_debut_jour(month)
        self.meshgrid_threshold()



    def data_retrieval_clustering(self):
        dict_agence = {}
        dict_agence["code_agence"] = self.agence
        result_nb = self.vals_seuil_nb_obs()   ## A changer (ce n'est pas la bonne fonction à regarder)
        result_quant = self.montants_obs_jour()
        quantiles_retraits = self.quantiles_retraits()
        result_quant_95 = self.count_freq_above(quantiles_retraits["quant_95"])
        result_quant_99 = self.count_freq_above(quantiles_retraits["quant_99"])
        dict_agence["Moy_nb_versements_j"] = result_nb["versements_j"]
        dict_agence["Moy_nb_retraits_j"] = result_nb["retraits_j"]
        dict_agence["Moy_versements_j"] = result_quant["moy_versements_j"]
        dict_agence["Moy_retraits_j"] = result_quant["moy_retraits_j"]
        dict_agence["Median_versements_j"] = result_quant["med_versements_j"]
        dict_agence["Median_retraits_j"] = result_quant["med_retraits_j"]
        dict_agence["Std_versements_j"] = result_quant["std_versements_j"]
        dict_agence["Std_retraits_j"] = result_quant["std_retraits_j"]
        dict_agence[f"Nb_clients_{self.year}"] = self.nb_clients()
        dict_agence["Nb_moy_transactions_j"] = result_nb["obs_j"]
        dict_agence["Nb_retraits_sup_quant_95"] = result_quant_95[0]
        dict_agence["Nb_retraits_sup_quant_99"] = result_quant_99[0]
        dict_agence["Freq_retraits_sup_quant_95"] = result_quant_95[0]
        dict_agence["Freq_retraits_sup_quant_99"] = result_quant_99[1]
        return dict_agence   # Permet de renvoyer la donnée utile pour le clustering pour l'agence considérée
  

Problèmes à gérer:

- L'échelle des plots ne correspond pas (entre les versements et les retraits)   
- Il faut inclure des méthodes concernant les 'outliers', en particulier le calcul de leur nombre et de leur fréquence.  #Ok (normalement)
- On peut aussi inclure des méthodes spécifiques pour les flux nets (variable d'intérêt pour la modélisation)